# Предобученная модель YOLO для детектинга людей  



In [1]:
from ultralytics import YOLO
import cv2

# Загрузка предобученной модели YOLOv10 (можно 'yolov10n.pt', 'yolov10s.pt' или yolov10m.pt)
model = YOLO('yolov10n.pt')  # nano-версия (быстрая, но менее точная)

model = YOLO('yolov10m.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data='my_dataset_yolo/data.yaml', epochs=100, imgsz=640, model='yolov8m.pt')

KeyboardInterrupt: 

In [ ]:
# Загрузка изображения
#image = cv2.imread('2.png')
image = cv2.imread('E://AIM/AI-Flow-Detecting/ai-core/data/2025-08-08_11-29-04.png')

# Детекция людей (класс '0' в COCO = человек)
results = model(image, classes=[0])  

# Визуализация результатов
annotated_image = results[0].plot()  # Рисует bounding boxes
cv2.imshow('Detected People', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Подсчёт количества людей
people_count = len(results[0].boxes)
print(f"Количество людей на фото: {people_count}")



0: 384x640 3 persons, 147.3ms
Speed: 4.6ms preprocess, 147.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
from ultralytics import YOLO
import cv2

# Загрузка модели YOLOv8 (nano — быстро, подходит для реального времени)
model = YOLO('yolov10n.pt')

# URL видеопотока (убраны лишние пробелы в конце!)
stream_url = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"

# Открываем видеопоток
cap = cv2.VideoCapture(stream_url)

if not cap.isOpened():
    print("❌ Ошибка: Не удалось открыть видеопоток. Проверь URL и соединение.")
    exit()

print("✅ Видеопоток запущен. Детекция людей через YOLOv8...")

while True:
    ret, frame = cap.read()

    if not ret:
        print("⚠️ Не удалось получить кадр. Поток может быть разорван.")
        break

    # Детекция ТОЛЬКО людей (класс 0 в COCO)
    results = model.predict(frame, classes=[0], conf=0.5, verbose=False)

    # Наносим bounding boxes и метки
    annotated_frame = results[0].plot()

    # Подсчёт обнаруженных людей
    people_count = len(results[0].boxes)
    cv2.putText(annotated_frame, f'Людей: {people_count}', (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2, cv2.LINE_AA)

    # Показываем кадр
    cv2.imshow('YOLOv8 — Детекция людей в реальном времени', annotated_frame)

    # Выход по клавише 'q' или пробелу
    if cv2.waitKey(1) & 0xFF in [ord('q'), ord(' ')]:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
print("⏹️ Работа завершена.")

✅ Видеопоток запущен. Детекция людей через YOLOv8...
⏹️ Работа завершена.


In [ ]:
import cv2
from ultralytics import YOLO
#plots
import matplotlib.pyplot as plt
import seaborn as sns

#basics
import pandas as pd
import numpy as np
import os
import subprocess

from tqdm.notebook import tqdm

# Display image and videos
import IPython
from IPython.display import Video, display
%matplotlib inline


import urllib.request 
import shutil

In [1]:
import cv2
import time
import os

os.makedirs("data/images", exist_ok=True)

cap = cv2.VideoCapture("https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8")
count = 0

print("Сбор кадров начат... Нажми 'q' для остановки.")

while True:
    ret, frame = cap.read()
    if ret:
        if count % 60 == 0:  # один кадр каждые 2 сек (30 FPS)
            cv2.imwrite(f"data/images/frame_{count}.jpg", frame)
            print(f"Сохранён кадр {count}")
        count += 1

    if cv2.waitKey(1) & 0xFF == ord('q') or count > 500:  # 500 кадров
        break

cap.release()
cv2.destroyAllWindows()

Сбор кадров начат... Нажми 'q' для остановки.
Сохранён кадр 0
Сохранён кадр 60
Сохранён кадр 120
Сохранён кадр 180
Сохранён кадр 240
Сохранён кадр 300
Сохранён кадр 360
Сохранён кадр 420
Сохранён кадр 480


In [2]:
import os
import pandas as pd

def convert_csv_to_yolo(csv_path, output_dir, class_name="passenger"):
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)
    
    df = pd.read_csv(csv_path)
    name_to_id = {class_name: 0}
    
    for _, row in df.iterrows():
        img_name = row['filename']
        w, h = row['width'], row['height']
        x_center = (row['xmin'] + row['xmax']) / 2 / w
        y_center = (row['ymin'] + row['ymax']) / 2 / h
        bbox_width = (row['xmax'] - row['xmin']) / w
        bbox_height = (row['ymax'] - row['ymin']) / h
        
        label_path = os.path.join(output_dir, "labels", os.path.splitext(img_name)[0] + ".txt")
        with open(label_path, "a") as f:
            f.write(f"{name_to_id[class_name]} {x_center} {y_center} {bbox_width} {bbox_height}\n")

# Пример:
# convert_csv_to_yolo("annotations.csv", "./data/train", "passenger")


In [ ]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict

DATA_YAML = "dataset/data.yaml"  # путь к вашему датасету
MODEL_PATH = "yolov8n-seg.pt"  # предобученная модель
VIDEO_PATH = "test.mp4"  # тестовое видео
SAVE_PATH = "output.mp4"

# ==============================
# 4. Обучение модели
# ==============================
model = YOLO(MODEL_PATH)
model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=8,
    device=0 if torch.cuda.is_available() else 'cpu'
)

# ==============================
# 5. Инициализация трекера
# ==============================
tracker = DeepSort(
    max_age=30,
    n_init=3,
    nms_max_overlap=1.0,
    max_cosine_distance=0.3,
    embedder="mobilenet",
    half=True,
)

# ==============================
# 6. Функция для проверки центра объекта
# ==============================
def is_in_center(bbox, frame_shape, center_zone=0.2):
    x1, y1, x2, y2 = bbox
    cx = (x1 + x2) / 2
    cy = (y1 + y2) / 2
    h, w = frame_shape[:2]
    cx_norm = cx / w
    cy_norm = cy / h
    return (
        0.5 - center_zone / 2 < cx_norm < 0.5 + center_zone / 2 and
        0.5 - center_zone / 2 < cy_norm < 0.5 + center_zone / 2
    )

# ==============================
# 7. Инференс и трекинг
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
out = cv2.VideoWriter(SAVE_PATH, cv2.VideoWriter_fourcc(*"mp4v"), 30,
                      (int(cap.get(3)), int(cap.get(4))))

counted_ids = set()
frame_idx = 0
passenger_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, imgsz=640, verbose=False)[0]

    detections = []
    for box, conf, cls in zip(results.boxes.xyxy.cpu().numpy(),
                              results.boxes.conf.cpu().numpy(),
                              results.boxes.cls.cpu().numpy()):
        if int(cls) == 0:  # класс "человек"
            detections.append(([box[0], box[1], box[2]-box[0], box[3]-box[1]], conf, "person"))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()

        if is_in_center(ltrb, frame.shape):
            if track_id not in counted_ids:
                counted_ids.add(track_id)
                passenger_count += 1

        x1, y1, x2, y2 = map(int, ltrb)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID:{track_id}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.putText(frame, f"Count: {passenger_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"Обработка завершена. Итоговое количество пассажиров: {passenger_count}")

# ==============================
# 8. Пример метрики IoU
# ==============================
def iou(mask1, mask2):
    inter = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return inter / union if union != 0 else 0




Ultralytics 8.3.176  Python-3.12.6 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

RuntimeError: Dataset 'data.yaml' error  'data.yaml' does not exist

In [13]:
# ==============================
# 9. Пример визуализации
# ==============================
img = cv2.imread("debug_train_test.jpg")  # тестовое изображение
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

# Путь к модели
model_path = "runs/segment/train/weights/best.pt"  # ваша обученная модель

# Инициализация YOLO
model = YOLO(model_path)

# Инициализация DeepSort
tracker = DeepSort(max_age=30)

# Источник: 0 — вебкамера, либо RTSP/HTTP/YouTube URL
stream_url = 0  # или "rtsp://user:pass@ip:port/stream"

cap = cv2.VideoCapture(stream_url)

if not cap.isOpened():
    print("Не удалось подключиться к потоку")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Детекция
    results = model(frame, imgsz=640, conf=0.5)

    detections = []
    for r in results:
        if hasattr(r, 'boxes'):
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls))

    # Трекинг
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        track_id = track.track_id
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Stream Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'runs\\segment\\train\\weights\\best.pt'

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv

# Загружаем обученную модель
model = YOLO("runs/segment/train/weights/best.pt")

# Трекер DeepSort
tracker = sv.ByteTrack()

video_path = "test_video.mp4"
cap = cv2.VideoCapture(video_path)

# Линия для подсчета
line_start = (100, 300)
line_end = (500, 300)
line_zone = sv.LineZone(start=line_start, end=line_end)
line_annotator = sv.LineZoneAnnotator(thickness=2, text_thickness=2, text_scale=1)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    
    detections = tracker.update_with_detections(detections)
    line_zone.trigger(detections)
    
    annotated_frame = frame.copy()
    line_annotator.annotate(annotated_frame, line_zone)
    
    cv2.imshow("Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np

def mean_absolute_error(true_counts, pred_counts):
    return np.mean(np.abs(np.array(true_counts) - np.array(pred_counts)))

def iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union != 0 else 0


In [3]:
# train.py
from ultralytics import YOLO
import torch

# Выбираем устройство
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используется: {device}")

# Загружаем предобученную модель
model = YOLO('yolov10n.pt')  # можно yolov8s.pt для точности

# Обучение
model.train(
    data='data.yaml',        # см. ниже
    epochs=100,
    imgsz=640,
    batch=16,
    device=device,
    name='yolov10n_bus_stop_finetuned',
    augment=True,
    patience=15,
    optimizer='AdamW',
    lr0=1e-3,
    iou=0.5,
    conf=0.25,
    project='runs/train'
)

# Сохранение
model.save('models/yolov10n_best.pt')
print("✅ Модель дообучена и сохранена.")

Используется: cpu
Ultralytics 8.3.176  Python-3.12.6 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov10n_bus_stop_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=15, perspective=

train: Scanning E:\AIM\AI-Flow-Detecting\ai-core\data\train\labels.cache... 0 images, 5 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]

WARNING Labels are missing or empty in E:\AIM\AI-Flow-Detecting\ai-core\data\train\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: Fast image access  (ping: 0.00.0 ms, read: 99.614.3 MB/s, size: 1070.0 KB)



e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning E:\AIM\AI-Flow-Detecting\ai-core\data\val\labels.cache... 0 images, 5 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]

WARNING Labels are missing or empty in E:\AIM\AI-Flow-Detecting\ai-core\data\val\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...


100 epochs completed in 0.054 hours.
Optimizer stripped from runs\train\yolov10n_bus_stop_finetuned\weights\last.pt, 5.7MB
Optimizer stripped from runs\train\yolov10n_bus_stop_finetuned\weights\best.pt, 5.7MB

Validating runs\train\yolov10n_bus_stop_finetuned\weights\best.pt...
Ultralytics 8.3.176  Python-3.12.6 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
YOLOv10n summary (fused): 102 layers, 2,265,363 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.07it/s]
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\ultralytics\utils\metrics.py:668: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\ultralytics\utils\metrics.py:713: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\numpy\_core\_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\ultralytics\utils\metrics.py:713: RuntimeWarning: Mean of empty slice.
  y = smoot

                   all          5          0          0          0          0          0
WARNING no labels found in detect set, can not compute metrics without labels
Speed: 0.6ms preprocess, 29.7ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\train\yolov10n_bus_stop_finetuned
✅ Модель дообучена и сохранена.


In [2]:
from ultralytics import YOLO
import cv2

# Загрузка модели YOLOv8 (nano — быстро, подходит для реального времени)
model = YOLO('models/yolov10n_best.pt')

# URL видеопотока (убраны лишние пробелы в конце!)
stream_url = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"

# Открываем видеопоток
cap = cv2.VideoCapture(stream_url)

if not cap.isOpened():
    print("❌ Ошибка: Не удалось открыть видеопоток. Проверь URL и соединение.")
    exit()

print("✅ Видеопоток запущен. Детекция людей через YOLOv8...")

while True:
    ret, frame = cap.read()

    if not ret:
        print("⚠️ Не удалось получить кадр. Поток может быть разорван.")
        break

    # Детекция ТОЛЬКО людей (класс 0 в COCO)
    results = model.predict(frame, classes=[0], conf=0.5, verbose=False)

    # Наносим bounding boxes и метки
    annotated_frame = results[0].plot()

    # Подсчёт обнаруженных людей
    people_count = len(results[0].boxes)
    cv2.putText(annotated_frame, f'Людей: {people_count}', (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2, cv2.LINE_AA)

    # Показываем кадр
    cv2.imshow('YOLOv8 — Детекция людей в реальном времени', annotated_frame)

    # Выход по клавише 'q' или пробелу
    if cv2.waitKey(1) & 0xFF in [ord('q'), ord(' ')]:
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()
print("⏹️ Работа завершена.")

✅ Видеопоток запущен. Детекция людей через YOLOv8...
⏹️ Работа завершена.


In [ ]:
# detect_on_stream.py
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import os
import subprocess

# -------------------------------
# ### Конфигурации
# -------------------------------
MODEL_PATH = 'runs/train/yolov10n_bus_stop_finetuned/weights/best.pt'  # твоя дообученная модель
STREAM_URL = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"
CONF_LEVEL = 0.6
SCALE_PERCENT = 100
VIDEO_NAME = "result.mp4"
ROI_POINTS = np.array([(1600, 750), (750, 450), (750, 600), (1700, 950)], np.int32)
ALPHA = 0.1
PATIENCE = 100
FRAME_MAX = 5
THR_CENTERS = 25

# -------------------------------
# Загрузка модели
# -------------------------------
model = YOLO(MODEL_PATH)
dict_classes = model.model.names

# -------------------------------
# Открываем поток
# -------------------------------
cap = cv2.VideoCapture(STREAM_URL)
if not cap.isOpened():
    print("❌ Не удалось открыть поток")
    exit()

# Параметры видео
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

if SCALE_PERCENT != 100:
    width = int(width * SCALE_PERCENT / 100)
    height = int(height * SCALE_PERCENT / 100)

# -------------------------------
# VideoWriter
# -------------------------------
tmp_out = "tmp_result.mp4"
output_out = "rep_result.mp4"

if os.path.exists(tmp_out): os.remove(tmp_out)
if os.path.exists(output_out): os.remove(output_out)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(tmp_out, fourcc, fps, (width, height))

# -------------------------------
# Вспомогательные функции
# -------------------------------
def resize_frame(frame, scale):
    if scale == 100: return frame
    w = int(frame.shape[1] * scale / 100)
    h = int(frame.shape[0] * scale / 100)
    return cv2.resize(frame, (w, h), interpolation=cv2.INTER_AREA)

def update_tracking(centers_old, center, thr, last_key, frame_idx, frame_max):
    best_id = None
    min_dist = float('inf')
    for obj_id, history in centers_old.items():
        last_center = list(history.keys())[-1]
        dist = np.linalg.norm(np.array(last_center) - np.array(center))
        if dist < thr and dist < min_dist:
            min_dist = dist
            best_id = obj_id

    if best_id is not None:
        centers_old[best_id][center] = frame_idx
        return centers_old, best_id, False, best_id
    else:
        new_id = f"p_{len(centers_old)}"
        centers_old[new_id] = {center: frame_idx}
        return centers_old, new_id, True, new_id

def filter_tracks(centers_old, patience):
    current_frame = max([max(t.values()) for t in centers_old.values()] + [0])
    to_remove = [k for k, v in centers_old.items() if current_frame - max(v.values()) > patience]
    for k in to_remove:
        del centers_old[k]
    return centers_old

# -------------------------------
# Основной цикл
# -------------------------------
centers_old = {}
total_count = 0
frame_idx = 0

print("✅ Обработка начата. Нажмите 'q' для остановки.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Кадр не получен. Переподключение...")
        cap.release()
        cap = cv2.VideoCapture(STREAM_URL)
        continue

    frame_idx += 1
    frame = resize_frame(frame, SCALE_PERCENT)
    display_frame = frame.copy()

    # --- ROI ---
    overlay = display_frame.copy()
    cv2.polylines(overlay, [ROI_POINTS], True, (255, 0, 0), 2)
    cv2.fillPoly(overlay, [ROI_POINTS], (255, 0, 0))
    cv2.addWeighted(overlay, ALPHA, display_frame, 1 - ALPHA, 0, display_frame)

    # Вырезаем ROI
    x_min, y_min = ROI_POINTS[:, 0].min(), ROI_POINTS[:, 1].min()
    x_max, y_max = ROI_POINTS[:, 0].max(), ROI_POINTS[:, 1].max()
    roi_frame = frame[y_min:y_max, x_min:x_max]

    # --- Детекция ---
    results = model(roi_frame, conf=CONF_LEVEL, classes=[0], verbose=False)
    if len(results[0].boxes) == 0:
        detections = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf'])
    else:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        conf = results[0].boxes.conf.cpu().numpy()
        detections = pd.DataFrame(np.column_stack([boxes, conf]),
                                  columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf'])

    # --- Обработка каждого человека ---
    for _, row in detections.iterrows():
        xmin, ymin, xmax, ymax, conf_val = map(int, row)
        cx = (xmin + xmax) // 2 + x_min
        cy = (ymin + ymax) // 2 + y_min

        centers_old, obj_id, is_new, _ = update_tracking(
            centers_old, (cx, cy), THR_CENTERS, None, frame_idx, FRAME_MAX
        )
        total_count += is_new

        # Рисуем
        cv2.rectangle(display_frame, (xmin + x_min, ymin + y_min), (xmax + x_min, ymax + y_min), (0, 255, 0), 2)
        cv2.circle(display_frame, (cx, cy), 5, (0, 255, 0), -1)
        cv2.putText(display_frame, f"{obj_id} ({conf_val:.2f})", (xmin + x_min, ymin + y_min - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    # Фильтрация
    centers_old = filter_tracks(centers_old, PATIENCE)

    # Счётчик
    cv2.putText(display_frame, f"Людей в зоне: {total_count}", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 2)

    # Запись и отображение
    out.write(display_frame)
    cv2.imshow("YOLOv8 — Пассажиропоток", display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# -------------------------------
# Финализация
# -------------------------------
out.release()
cap.release()
cv2.destroyAllWindows()

# Перекодировка
subprocess.run([
    "ffmpeg", "-i", tmp_out,
    "-crf", "18", "-preset", "veryfast",
    "-vcodec", "libx264", output_out
], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

os.remove(tmp_out)
print(f"✅ Видео сохранено: {output_out}")

❌ Не удалось открыть поток
✅ Обработка начата. Нажмите 'q' для остановки.
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподключение...
⚠️ Кадр не получен. Переподкл

FileNotFoundError: [WinError 2] Не удается найти указанный файл

: 

In [1]:
result = model(roi_frame, conf=CONF_LEVEL, classes=[0], verbose=False)
print("Все классы в этом кадре:", results[0].boxes.cls.cpu().numpy())
print("Все имена:", dict_classes)

NameError: name 'model' is not defined

In [2]:
# train.py
from ultralytics import YOLO
import torch
import os

# Папки
os.makedirs("models", exist_ok=True)
os.makedirs("runs/train", exist_ok=True)

# Устройство
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Используется устройство: {device}")

# Проверка data.yaml
data_yaml = """
train: data/images
val: data/images

nc: 1
names: ['person']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml.strip())

print("📄 data.yaml создан/обновлён.")

# Загрузка модели
model = YOLO('yolov10n.pt')  # или yolov8n.pt

# Обучение
print("🎯 Начинаем обучение...")
model.train(
    data='data.yaml',
    epochs=50,
    imgsz=640,
    batch=8 if device == 'cpu' else 16,
    device=device,
    name='yolov10n_person_finetune',
    augment=True,
    patience=15,
    optimizer='AdamW',
    lr0=1e-3,
    iou=0.5,
    conf=0.25,
    project='runs/train',
    save_period=10
)

# Сохранение
best_model_path = 'runs/train/yolov10n_person_finetune/weights/best.pt'
if os.path.exists(best_model_path):
    model.save('models/yolov10n_best.pt')
    print("✅ Модель сохранена: models/yolov10n_best.pt")
else:
    print("❌ Ошибка: best.pt не найден. Проверьте процесс обучения.")

# Тест на одном кадре (опционально)
test_img = "data/images/frame_0.jpg"
if os.path.exists(test_img):
    results = model(test_img, conf=0.1)
    for r in results:
        r.save(filename="debug_train_test.jpg")
    print("🖼 Тест на кадре: debug_train_test.jpg")

🚀 Используется устройство: cpu
📄 data.yaml создан/обновлён.
🎯 Начинаем обучение...
Ultralytics 8.3.176  Python-3.12.6 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov10n_person_finetune, nbs=64, nms=False, opset=None, optimize=False, o

train: Scanning E:\AIM\AI-Flow-Detecting\ai-core\data\labels... 0 images, 9 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<00:00, 817.82it/s]

WARNING train: No labels found in E:\AIM\AI-Flow-Detecting\ai-core\data\labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
train: New cache created: E:\AIM\AI-Flow-Detecting\ai-core\data\labels.cache
WARNING Labels are missing or empty in E:\AIM\AI-Flow-Detecting\ai-core\data\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: Fast image access  (ping: 0.00.0 ms, read: 3097.1852.9 MB/s, size: 960.7 KB)



e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning E:\AIM\AI-Flow-Detecting\ai-core\data\labels.cache... 0 images, 9 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<?, ?it/s]

WARNING Labels are missing or empty in E:\AIM\AI-Flow-Detecting\ai-core\data\labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...


50 epochs completed in 0.050 hours.
Optimizer stripped from runs\train\yolov10n_person_finetune\weights\last.pt, 5.7MB
Optimizer stripped from runs\train\yolov10n_person_finetune\weights\best.pt, 5.7MB

Validating runs\train\yolov10n_person_finetune\weights\best.pt...
Ultralytics 8.3.176  Python-3.12.6 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
YOLOv10n summary (fused): 102 layers, 2,265,363 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Model does not support 'augment=True', reverting to single-scale prediction.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\ultralytics\utils\metrics.py:668: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\numpy\_core\_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\ultralytics\utils\metrics.py:713: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\numpy\_core\_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
e:\AIM\AI-Flow-Detecting\ai-core\mai\Lib\site-packages\ultralytics\utils\metrics.py:713: RuntimeWarning: Mean of empty slice.
  y = smoot

                   all          9          0          0          0          0          0
WARNING no labels found in detect set, can not compute metrics without labels
Speed: 0.8ms preprocess, 38.9ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\train\yolov10n_person_finetune
✅ Модель сохранена: models/yolov10n_best.pt

WARNING Model does not support 'augment=True', reverting to single-scale prediction.
image 1/1 e:\AIM\AI-Flow-Detecting\ai-core\data\images\frame_0.jpg: 384x640 (no detections), 43.0ms
Speed: 1.8ms preprocess, 43.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
🖼 Тест на кадре: debug_train_test.jpg


In [3]:
# detect_on_stream.py
from ultralytics import YOLO
import cv2
import numpy as np
import pandas as pd
import os
import subprocess
import time

# -------------------------------
# ### Конфигурация
# -------------------------------
MODEL_PATH = 'models/yolov10n_best.pt'  # или: runs/train/yolov10n_person_finetune/weights/best.pt
STREAM_URL = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"
CONF_LEVEL = 0.3  # понижено для отладки
SCALE_PERCENT = 100
VIDEO_NAME = "result.mp4"
ALPHA = 0.1
PATIENCE = 30  # кадров без обновления
FRAME_MAX = 5
THR_CENTERS = 40  # порог сопоставления центров

# 🟡 ROI — прямоугольник (проще и надёжнее). Замените координаты под вашу камеру.
# Используйте `debug_roi.py` ниже, чтобы подобрать.
ROI_RECT = (600, 400, 1800, 1000)  # (x1, y1, x2, y2) — область интереса

# -------------------------------
# Проверка модели
# -------------------------------
if not os.path.exists(MODEL_PATH):
    print(f"❌ Модель не найдена: {MODEL_PATH}")
    print("Проверьте путь или запустите train.py")
    exit()

model = YOLO(MODEL_PATH)
dict_classes = model.model.names
print(f"📋 Классы модели: {dict_classes}")

# Убедимся, что класс 'person' есть
person_class_id = None
for k, v in dict_classes.items():
    if 'person' in v.lower():
        person_class_id = k
        break

if person_class_id is None:
    print("❌ Не найден класс 'person' в модели. Проверьте data.yaml и обучение.")
    exit()

print(f"✅ Класс 'person' найден с ID={person_class_id}")

# -------------------------------
# Открываем поток
# -------------------------------
cap = cv2.VideoCapture(STREAM_URL)
if not cap.isOpened():
    print("❌ Не удалось открыть поток. Проверьте URL.")
    exit()

# Параметры видео
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 25

if SCALE_PERCENT != 100:
    width = int(width * SCALE_PERCENT / 100)
    height = int(height * SCALE_PERCENT / 100)

# -------------------------------
# VideoWriter
# -------------------------------
tmp_out = "tmp_result.mp4"
output_out = "rep_result.mp4"

for f in [tmp_out, output_out]:
    if os.path.exists(f):
        os.remove(f)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(tmp_out, fourcc, fps, (width, height))

# -------------------------------
# Вспомогательные функции
# -------------------------------
def resize_frame(frame, scale):
    if scale == 100:
        return frame
    w = int(frame.shape[1] * scale / 100)
    h = int(frame.shape[0] * scale / 100)
    return cv2.resize(frame, (w, h), interpolation=cv2.INTER_AREA)

def update_tracking(centers_old, center, thr, frame_idx):
    best_id = None
    min_dist = float('inf')
    for obj_id, history in centers_old.items():
        last_center = list(history.keys())[-1]
        dist = np.linalg.norm(np.array(last_center) - np.array(center))
        if dist < thr and dist < min_dist:
            min_dist = dist
            best_id = obj_id

    if best_id is not None:
        centers_old[best_id][center] = frame_idx
        return centers_old, best_id, False
    else:
        new_id = f"p_{len(centers_old)}"
        centers_old[new_id] = {center: frame_idx}
        return centers_old, new_id, True

def filter_tracks(centers_old, patience, current_frame):
    to_remove = [k for k, v in centers_old.items() if current_frame - max(v.values()) > patience]
    for k in to_remove:
        del centers_old[k]
    return centers_old

# -------------------------------
# Основной цикл
# -------------------------------
centers_old = {}
total_count = 0
frame_idx = 0

print("✅ Обработка начата. Нажмите 'q' для остановки.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Кадр не получен. Переподключение...")
        time.sleep(1)
        cap.release()
        cap = cv2.VideoCapture(STREAM_URL)
        continue

    frame_idx += 1
    frame = resize_frame(frame, SCALE_PERCENT)
    display_frame = frame.copy

📋 Классы модели: {0: 'person'}
✅ Класс 'person' найден с ID=0
✅ Обработка начата. Нажмите 'q' для остановки.


KeyboardInterrupt: 

In [4]:
# debug_roi.py
import cv2

cap = cv2.VideoCapture("https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8")

print("Нажмите 's' чтобы зафиксировать кадр и подобрать ROI. 'q' — выход.")

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    cv2.imshow("Подбор ROI", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        x, y, w, h = cv2.selectROI("ROI", frame, False)
        print(f"ROI_RECT = ({x}, {y}, {x+w}, {y+h})")
        cv2.destroyWindow("ROI")
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Нажмите 's' чтобы зафиксировать кадр и подобрать ROI. 'q' — выход.


KeyboardInterrupt: 

In [ ]:
# Checking samples of processed frames
for i in [62,63, 64, 65, 66]:
    plt.figure(figsize =( 14, 10))
    plt.imshow(frames_list[i])
    plt.show()

In [ ]:
#output video result
frac = 0.7 
Video(data='rep_result.mp4', embed=True, height=int(720 * frac), width=int(1280 * frac))